# Analysis

This notebook formats raw data and creates data for map products

In [1]:
import os
import sys
import arcpy
import pandas as pd
import numpy as np
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
from pathlib import Path

In [2]:
# Common paths
dir_project = Path.cwd().parent
dir_data = dir_project/'data'
dir_raw = dir_data/'raw'
dir_processed = dir_data/'processed'
gdb_processed = dir_processed/'processed.gdb'
aprx_name = 'SDR_WQIP_AnnualReport.aprx'

aprx = arcpy.mp.ArcGISProject(dir_project/'arcgis'/aprx_name)

## Appendix C1 Bioassessment

- Arcgis maps require two datasets for producing appendix C1 bioassessment maps
  - bio_stations, current year stations and results
  - bio_trends, all stations and results
- Data taken from "SDR Bioassement Tab and Figs (ABC Labs) 2022 (v3) (002).xlsx" recieved 01/05/2023
  - Current year stations from Tab C-11.4 were appended to "Bioassessment Stations.xlsx" list of all stations
  - Added StationType to sheet "Tab C-11.4" from teams message
  - All instances of "ALC04@Aliso Park" were replaced to "ALC04@AlisoPark"



#### Function Definitions

In [3]:
# Function definitions
def create_stations(raw_data: dict, key: str = 'Tab C-11.4'):
    '''Creates spatial data from spreadsheet coordinates

    Parameters
        raw_data: Dictionary of excel sheets from pandas.read_excel
        key: Sheet name

    Returns
        spatial data frame
    '''

    df = (
        raw_data[key]
        .loc[0:4]
        .rename(columns=raw_data[key].iloc[0])
        .loc[1:]
        .iloc[:, [1, 2, 3, 4, 5, 6]]
        .rename(
            columns=lambda col: 'StationDescription'
            if col == 'Station Description'
            else col
        )
    )
    return pd.DataFrame.spatial.from_xy(
        df=df, x_column='Longitude', y_column='Latitude', sr=4326
    )


def create_stations_historic(raw_data: pd.DataFrame):
    '''Creates spatial data for all stations combining current data with historic stations

    Parameters
        raw_data: Data Frame

    Returns
        pandas data frame
    '''
    return pd.DataFrame.spatial.from_xy(
        df=raw_data, x_column='Longitude', y_column='Latitude', sr=4326
    )


def add_current_scores(
    raw_data: dict,
    csci_buckets: list = [0, 0.62, 0.79, 0.92, 1],
    cram_buckets: list = [0, 43, 62, 81, 100],
    colors: list = ['Red', 'Orange', 'Yellow', 'Blue'],
    key: str = 'CRAM CSCI Correlation',
):
    '''Add the CSCI and CRAM data to the current monitoring year stations, results are binned

    Parameters
        raw_data: Dictionary of excel sheets from pandas.read_excel
        csci_buckets: Bins for CSCI symbology
        cram_buckets: Bins for CRAM symbology
        colors: Text color for bins
        key: Sheet name with data

    Returns
        pandas data frame
    '''

    df = create_stations(raw_data).merge(
        raw_data[key]
        .iloc[1:, [0, 1, 2, 3]]
        .rename(columns=raw_data[key].iloc[0])
        .rename(columns={"Overall CRAM Score": "CRAMScore", "CSCI Score": "CSCIScore"})
        .sort_values('Year')
        .groupby('Station')
        .tail(1),
        on=['Station'],
        how='left',
    )

    df['CSCIColor'] = pd.cut(x=df['CSCIScore'], bins=csci_buckets, labels=colors)
    df['CRAMColor'] = pd.cut(x=df['CRAMScore'], bins=cram_buckets, labels=colors)

    return df.assign(
        **{c: lambda df_, c=c: df_[c].astype('str') for c in ['CRAMColor', 'CSCIColor']}
    )


def add_mean_scores(
    stations: pd.DataFrame,
    raw_data: dict,
    csci_buckets: list = [0, 0.62, 0.79, 0.92, np.inf],
    cram_buckets: list = [0, 43, 62, 81, 100],
    colors: list = ['Red', 'Orange', 'Yellow', 'Blue'],
    h20_buckets: list = [0, 19, 34, 48, 57, np.inf],
):
    '''Adds the mean historic data and trend data to stations

    Parameters
        raw_data: Dictionary of excel sheets from pandas.read_excel
        csci_buckets: Bins for CSCI symbology
        cram_buckets: Bins for CRAM symbology
        colors: Text color for bins
        h20_buckets: Bins for H2O symbology

    Returns
        spatial data frame
    '''

    correlation_key = 'CRAM CSCI Correlation'
    trend_key = 'OCPW SDR Data 2009-2022'

    cram = (
        raw_data[correlation_key]
        .rename(columns=raw_data[correlation_key].iloc[0])
        .loc[1:]
        .iloc[:, [0, 1, 2, 3]]
        .rename(columns=lambda col: col.replace(' ', ''))
        .drop(columns=['Year'])
        .assign(OverallCRAMScore=lambda x: x["OverallCRAMScore"].astype(float))
        .groupby(['Station'])
        .mean()
    )

    cram['CRAMColor'] = pd.cut(
        x=cram['OverallCRAMScore'], bins=cram_buckets, labels=colors
    )

    # Fix for 2009 TC-AP station that was in close proximity to SMC00206
    trends = raw_data[trend_key]

    trends['Station'] = np.where(
        trends['Station Unique'] == 'TC-AP-2009', 'SMC00206', trends['Station']
    )
    # End fix

    csci_h20 = (
        trends.iloc[:, [0, 1, 2, 3, 9, 11]]
        .drop(columns=['Cluster Group', 'Year'])
        .groupby(['Station', 'Parameter'])
        .mean()
        .reset_index()
        .pivot(index='Station', columns='Parameter', values='Result')
    )

    csci_h20['CSCIColor'] = pd.cut(x=csci_h20['CSCI'], bins=csci_buckets, labels=colors)
    csci_h20['H20Color'] = pd.cut(x=csci_h20['H20'], bins=h20_buckets, labels=['Red', 'Orange', 'Yellow', 'Green', 'Blue'])

    cluster = (
        trends.sort_values(by='Year')
        .groupby('Station')[['Cluster Group', 'Year']]
        .last()
    )

    output = (
        create_stations_historic(stations)
        .merge(cram, on=['Station'], how='left')
        .merge(csci_h20, on=['Station'], how='left')
        .merge(cluster, on=['Station'], how='left')
        .rename(columns=lambda col: col.replace(' ', ''))
        .rename(columns=lambda col: col.replace('_', ''))
        .assign(
            **{
                c: lambda df_, c=c: df_[c].astype('str')
                for c in ['CRAMColor', 'H20Color', 'CSCIColor']
            }
        )  # python api does not like category dyptes
    )
    return output


def format_bioassessment(stations: pd.DataFrame, raw_data: dict, output_gdb: str):
    '''Calls formatting functions and saves to gdb'''

    current_year = add_current_scores(raw_data)
    trend_data = add_mean_scores(stations, raw_data)

    current_year.spatial.to_featureclass(
        location=str(gdb_processed / 'bio_stations'), overwrite=True
    )
    trend_data.spatial.to_featureclass(
        location=str(gdb_processed / 'bio_trends'), overwrite=True
    )

    return [current_year, trend_data]

#### Analysis

In [4]:
# Import raw data
abc_data = pd.read_excel(dir_raw/'SDR  Bioassement Tab and Figs (ABC Labs) 2022 (v3) (002).xlsx', sheet_name=None)
all_stations = pd.read_excel(dir_raw/'Bioassessment Stations.xlsx', sheet_name='Sheet1')


In [5]:
# Analyze and save data to geodatabase
results = format_bioassessment(all_stations, abc_data, gdb_processed)

In [6]:
results[0]

,Station,StationDescription,Sample Date,Latitude,Longitude,StationType,SHAPE,Year,CRAMScore,CSCIScore,CSCIColor,CRAMColor
0,901M14169,Wood Canyon,2022-05-19 00:00:00,33.576139,-117.747691,SMC,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",2022,72,0.618818,Red,Yellow
1,901M14170,Arroyo Trabuco,2022-06-08 00:00:00,33.56243,-117.652048,SMC,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",2022,81,0.928795,Blue,Yellow
2,ALC04@AlisoPark,Aliso Creek,2022-05-18 00:00:00,33.60125,-117.705061,Pre-Restoration,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",2022,64,0.796364,Yellow,Yellow
3,SMC00531,Laguna Canyon Wash,2022-04-19 00:00:00,33.55975,-117.772,SMC,"{""spatialReference"": {""wkid"": 4326}, ""x"": -117...",2022,31,0.42134,Red,Red


### Appendix B2 OCFS Maps

Maps not created in 2021-22

#### Function Definitions


In [84]:
def format_ocfs(raw_data: dict, output_gdb: str, key: str = "Data"):

    return (
        GeoAccessor.from_xy(raw_data[key], "Long", "Lat").spatial.to_featureclass(
            f'{output_gdb}\\ocfs_stations')
    )


In [85]:
ocfs_data = pd.read_excel(
    dir_raw/'OCFS Coordinates.xlsx', sheet_name=None)

# format_ocfs(ocfs_data, gdb_processed)


'h:\\ProjectsH\\Github\\WQIP-Annual-Report-2020-21-Maps\\data\\processed\\processed.gdb\\ocfs_stations'